# Visual Design Matrix (VDM) Generator

Creates VDM files from pRF stimulus videos.

**Specifications:**
- Input: 102×102 videos at 60 Hz
- Output: 50×50 downsampled VDM
- Sampling: Every 2 seconds (120 frames at 60 Hz)
- Output formats: .npy and .mp4
- No additional padding (already done in input videos)

In [1]:
# General imports
import os
import cv2
import numpy as np

In [2]:
# Define parameters
input_dir = '/home/mszinte/disks/meso_S/data/amblyo7T_prf/derivatives/vdm/original/'
output_dir = '/home/mszinte/disks/meso_S/data/amblyo7T_prf/derivatives/vdm/'

# VDM settings
vdm_width = 50
vdm_height = 50
TR = 2  # seconds - sampling rate

# Input videos to process
video_files = [
    'Bars_60hz.mp4',
    'Wedges_60hz.mp4',
    'Rings_60hz.mp4',
    'Bars_Bars_Rings_Wedges_60hz.mp4'
]

# Corresponding output names
output_names = [
    'Bars',
    'Wedges',
    'Rings',
    'pRF'
]

In [3]:
# Process each video file
for video_file, task_name in zip(video_files, output_names):
    print(f"\n=== Processing {video_file} ===")
    
    # Open video
    stim_video_fn = os.path.join(input_dir, video_file)
    cap = cv2.VideoCapture(stim_video_fn)
    
    if not cap.isOpened():
        print(f"  ✗ Could not open {video_file}")
        continue
    
    # Get video settings
    vid_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    vid_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    vid_fps = cap.get(cv2.CAP_PROP_FPS)
    vid_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"  Video: {vid_width}×{vid_height}, {vid_fps} fps, {vid_frames} frames")
    
    # Calculate VDM dimensions
    vdm_frames_sampled = vid_fps * TR  # 120 frames per sample
    vdm_frames = int(vid_frames / vdm_frames_sampled)
    vdm_mat = np.zeros((vdm_width, vdm_height, vdm_frames))
    
    print(f"  VDM: {vdm_width}×{vdm_height}, {vdm_frames} timepoints")
    print(f"  Sampling every {int(vdm_frames_sampled)} frames ({TR} seconds)")
    
    # Create VDM matrix
    vid_idx, vdm_idx = 0, 0
    
    while cap.isOpened():
        # Read video frames
        ret, vid_frame = cap.read()
        if not ret:
            break
        
        # Process sampled frames (every TR seconds)
        if np.mod(vid_idx, int(vdm_frames_sampled)) == 0:
            # Convert frame to grayscale
            gray_mat = cv2.cvtColor(vid_frame, cv2.COLOR_BGR2GRAY)
            
            # Convert to binary (threshold > 5)
            binary_mat = (gray_mat > 5).astype(np.uint8)
            
            # Resize to create VDM (no padding needed - already square)
            binary_resize_mat = cv2.resize(binary_mat, dsize=(vdm_width, vdm_height),
                                          interpolation=cv2.INTER_NEAREST)
            
            # Fill VDM matrix
            vdm_mat[..., vdm_idx] = binary_resize_mat
            vdm_idx += 1
            
            # Progress indicator
            if vdm_idx % 50 == 0:
                print(f"  Progress: {vdm_idx}/{vdm_frames} timepoints processed")
        
        vid_idx += 1
    
    cap.release()
    
    print(f"  Processed {vdm_idx} timepoints")
    
    # Save VDM as numpy matrix
    vdm_numpy_fn = os.path.join(output_dir, f'vdm_{task_name}_{vdm_width}_{vdm_height}.npy')
    np.save(vdm_numpy_fn, vdm_mat)
    print(f"  ✓ Saved {os.path.basename(vdm_numpy_fn)}")
    
    # Save VDM as video
    vdm_video_fn = os.path.join(output_dir, f'vdm_{task_name}_{vdm_width}_{vdm_height}.mp4')
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(vdm_video_fn, fourcc, 1/TR, (vdm_width, vdm_height), False)
    
    # Write each VDM frame
    for frame in np.split(vdm_mat, vdm_frames, axis=2):
        out.write(np.uint8(frame.squeeze() * 255))
    
    out.release()
    print(f"  ✓ Saved {os.path.basename(vdm_video_fn)}")

print("\n=== All VDM files created successfully! ===")


=== Processing Bars_60hz.mp4 ===
  Video: 102×102, 60.0 fps, 14400 frames
  VDM: 50×50, 120 timepoints
  Sampling every 120 frames (2 seconds)
  Progress: 50/120 timepoints processed
  Progress: 100/120 timepoints processed
  Processed 120 timepoints
  ✓ Saved vdm_Bars_50_50.npy
  ✓ Saved vdm_Bars_50_50.mp4

=== Processing Wedges_60hz.mp4 ===
  Video: 102×102, 60.0 fps, 14400 frames
  VDM: 50×50, 120 timepoints
  Sampling every 120 frames (2 seconds)
  Progress: 50/120 timepoints processed
  Progress: 100/120 timepoints processed
  Processed 120 timepoints
  ✓ Saved vdm_Wedges_50_50.npy
  ✓ Saved vdm_Wedges_50_50.mp4

=== Processing Rings_60hz.mp4 ===
  Video: 102×102, 60.0 fps, 14400 frames
  VDM: 50×50, 120 timepoints
  Sampling every 120 frames (2 seconds)
  Progress: 50/120 timepoints processed
  Progress: 100/120 timepoints processed
  Processed 120 timepoints
  ✓ Saved vdm_Rings_50_50.npy
  ✓ Saved vdm_Rings_50_50.mp4

=== Processing Bars_Bars_Wedges_Rings_60hz.mp4 ===
  Video: 